In [1]:
 current_dir = "gdrive/My Drive/Colab Notebooks/Thesis/gold_standard"


import os
import glob
from google.colab import drive
import pandas as pd
import re
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
from sklearn.preprocessing import MinMaxScaler

# Mount Google Drive
drive.mount('/content/gdrive')


Mounted at /content/gdrive


In [2]:
os.chdir(current_dir)

## Enrichment

In [3]:
df = pd.read_pickle("gold_psy.pkl")

In [4]:
group_dict = {'ridgeway': 'handbook',
 'bundy': 'handbook',
 'kemper': 'handbook',
 'gacy': 'handbook',
 'dahmer': 'handbook',
 'bernardo': 'cbc',
 'mitchell': 'articles',
 'marsh': 'articles',
 'wayne': 'doc',
 'carl': 'doc'}

In [5]:
df

name  score                                               text
0     wayne   40.0  \nmy very first contemplation of criminal beha...
1      carl   36.0  I had a statutory rape on a minor fifteen uh t...
2     bundy   34.0  \nyes that's true\n\ni grew up in a wonderful ...
3  bernardo   35.0  \npaul jason teal aka paul bernardo\n\n\nyeah ...
4    dahmer   23.0  \nhey you're lucky you came up on a\n\n\nday w...
5      gacy   27.0  \n\nwell that's hogwash yeah that's strictly h...
6    kemper   26.0  \nwell i'm not an expert i'm not an authority ...
7     marsh   34.5  \nevery time i look at someone i see flashes o...
8  mitchell   34.0  that would be nice i'd like to use the restroo...
9  ridgeway   19.0  \ni didn't really think i had a a urge to kill...

In [6]:
import re

def clean_text(text):
    # Replace newline characters with a space
    text = text.replace('\n', ' ')

    # Replace multiple spaces with a single space
    text = re.sub(r'\s+', ' ', text)

    # Remove non-alphanumeric characters (but keep single whitespaces)
    text = re.sub(r'[^a-zA-Z0-9 ]', '', text)

    # Convert text to lowercase
    return text.lower()


# Example usage

df['clean_text'] = df['text'].apply(clean_text)

In [7]:
def two_tier_score(score):
    if score > 30:
        return "psychopathic"
    else:
        return "not psychopathic"

# Apply the function to the 'Score' column
df['binary'] = df['score'].apply(two_tier_score)

In [ ]:
def three_tier_score(score):
    if score >= 30:
        return "high"
    elif 21 <= score < 29:
        return "medium"
    else:
        return "low"

# Apply the function to the 'score' column
df['trinary'] = df['score'].apply(three_tier_score)


In [18]:
def select_rows_by_group(group_name):
    return df[df['group'] == group_name]
df['group'] = df['name'].map(group_dict)

In [16]:
import spacy

def tag_texts(texts, model='en_core_web_sm'):
    """
    Tags a list of texts using SpaCy.

    :param texts: List of texts to be tagged.
    :param model: SpaCy language model to use.
    :return: List of tagged documents.
    """
    # Load the SpaCy model
    nlp = spacy.load(model)

    # Process each text and store the result
    tagged_texts = [nlp(text) for text in texts]

    return tagged_texts



# To see the tags of the first sentence



[Hello]


In [22]:
import spacy

def extract_pronouns(text, model='en_core_web_sm'):
    """
    Extracts pronouns from a given text using SpaCy.

    :param text: The text to process.
    :param model: The SpaCy language model to use.
    :return: A list of pronouns found in the text.
    """
    # Load the SpaCy model
    nlp = spacy.load(model)

    # Process the text
    doc = nlp(text)

    # Extract pronouns
    pronouns = [token.text for token in doc if token.pos_ == 'PRON']

    return " ".join(pronouns)



In [24]:
df["pronouns"] = df["clean_text"].apply(extract_pronouns)

name  score                                               text  \
0     wayne   40.0  \nmy very first contemplation of criminal beha...   
1      carl   36.0  I had a statutory rape on a minor fifteen uh t...   
2     bundy   34.0  \nyes that's true\n\ni grew up in a wonderful ...   
3  bernardo   35.0  \npaul jason teal aka paul bernardo\n\n\nyeah ...   
4    dahmer   23.0  \nhey you're lucky you came up on a\n\n\nday w...   
5      gacy   27.0  \n\nwell that's hogwash yeah that's strictly h...   
6    kemper   26.0  \nwell i'm not an expert i'm not an authority ...   
7     marsh   34.5  \nevery time i look at someone i see flashes o...   
8  mitchell   34.0  that would be nice i'd like to use the restroo...   
9  ridgeway   19.0  \ni didn't really think i had a a urge to kill...   

                                          clean_text            binary  \
0   my very first contemplation of criminal behav...      psychopathic   
1  i had a statutory rape on a minor fifteen uh t...      psychopathic   
2   yes thats true i grew up in a wonderful home ...      psychopathic   
3   paul jason teal aka paul bernardo yeah im and...      psychopathic   
4   hey youre lucky you came up on a day when the...  not psychopathic   
5   well thats hogwash yeah thats strictly hogwas...  not psychopathic   
6   well im not an expert im not an authority im ...  not psychopathic   
7   every time i look at someone i see flashes of...      psychopathic   
8  that would be nice id like to use the restroom...      psychopathic   
9   i didnt really think i had a a urge to kill h...  not psychopathic   

      group                                           pronouns  
0       doc  my i my i his he he he me you my you he i his ...  
1       doc                 i i a she its me it my i i i who i  
2  handbook  that i i i what i i i you we our whatever they...  
3       cbc  i it what that you i i there any i i they they...  
4  handbook  you you there it nothing nothing i my it me it...  
5  handbook  that that i it i i i i this i its i my it they...  
6  handbook  i i i someone who it its its it it they he she...  
7  articles  i someone i me them i it its what my i them i ...  
8  articles  that i i i i i my that i you you your its its ...  
9  handbook  i i her i her what i her her i her me she it i...

## Experiments

In [26]:
copy = df

In [27]:
copy

name  score                                               text  \
0     wayne   40.0  \nmy very first contemplation of criminal beha...   
1      carl   36.0  I had a statutory rape on a minor fifteen uh t...   
2     bundy   34.0  \nyes that's true\n\ni grew up in a wonderful ...   
3  bernardo   35.0  \npaul jason teal aka paul bernardo\n\n\nyeah ...   
4    dahmer   23.0  \nhey you're lucky you came up on a\n\n\nday w...   
5      gacy   27.0  \n\nwell that's hogwash yeah that's strictly h...   
6    kemper   26.0  \nwell i'm not an expert i'm not an authority ...   
7     marsh   34.5  \nevery time i look at someone i see flashes o...   
8  mitchell   34.0  that would be nice i'd like to use the restroo...   
9  ridgeway   19.0  \ni didn't really think i had a a urge to kill...   

                                          clean_text            binary  \
0   my very first contemplation of criminal behav...      psychopathic   
1  i had a statutory rape on a minor fifteen uh t...      psychopathic   
2   yes thats true i grew up in a wonderful home ...      psychopathic   
3   paul jason teal aka paul bernardo yeah im and...      psychopathic   
4   hey youre lucky you came up on a day when the...  not psychopathic   
5   well thats hogwash yeah thats strictly hogwas...  not psychopathic   
6   well im not an expert im not an authority im ...  not psychopathic   
7   every time i look at someone i see flashes of...      psychopathic   
8  that would be nice id like to use the restroom...      psychopathic   
9   i didnt really think i had a a urge to kill h...  not psychopathic   

      group                                           pronouns  
0       doc  my i my i his he he he me you my you he i his ...  
1       doc                 i i a she its me it my i i i who i  
2  handbook  that i i i what i i i you we our whatever they...  
3       cbc  i it what that you i i there any i i they they...  
4  handbook  you you there it nothing nothing i my it me it...  
5  handbook  that that i it i i i i this i its i my it they...  
6  handbook  i i i someone who it its its it it they he she...  
7  articles  i someone i me them i it its what my i them i ...  
8  articles  that i i i i i my that i you you your its its ...  
9  handbook  i i her i her what i her her i her me she it i...

In [30]:
print(df[['clean_text', 'pronouns']])

                                          clean_text  \
0   my very first contemplation of criminal behav...   
1  i had a statutory rape on a minor fifteen uh t...   
2   yes thats true i grew up in a wonderful home ...   
3   paul jason teal aka paul bernardo yeah im and...   
4   hey youre lucky you came up on a day when the...   
5   well thats hogwash yeah thats strictly hogwas...   
6   well im not an expert im not an authority im ...   
7   every time i look at someone i see flashes of...   
8  that would be nice id like to use the restroom...   
9   i didnt really think i had a a urge to kill h...   

                                            pronouns  
0  my i my i his he he he me you my you he i his ...  
1                 i i a she its me it my i i i who i  
2  that i i i what i i i you we our whatever they...  
3  i it what that you i i there any i i they they...  
4  you you there it nothing nothing i my it me it...  
5  that that i it i i i i this i its i my it they... 

## Binary - LOOCV

In [36]:
import pandas as pd
from sklearn.model_selection import LeaveOneOut
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.feature_extraction.text import CountVectorizer

# Initialize LeaveOneOut
loocv = LeaveOneOut()

# Prepare the classifier
clf = LogisticRegression(max_iter=1000)  # Increase max_iter if needed

# Convert text data to numerical data
vectorizer = CountVectorizer()

import pandas as pd

# Concatenate the text columns
df['combined_text'] = df['clean_text'] + " " + df['pronouns']

# Now you can vectorize this combined column
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(df['combined_text'])

y = df['binary']

# Store the accuracy for each fold and predictions
accuracies = []
all_y_test = []
all_y_pred = []

for train_index, test_index in loocv.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Train the model
    clf.fit(X_train, y_train)

    # Make predictions
    y_pred = clf.predict(X_test)

    # Store true labels and predictions
    all_y_test.extend(y_test)
    all_y_pred.extend(y_pred)

    # Calculate accuracy
    accuracy = accuracy_score(y_test, y_pred)
    accuracies.append(accuracy)

# Calculate the average accuracy
average_accuracy = sum(accuracies) / len(accuracies)
print(f"Average Accuracy: {average_accuracy}")

# Generate classification report
print("Classification Report:\n")
print(classification_report(all_y_test, all_y_pred))




Average Accuracy: 0.8
Classification Report:

                  precision    recall  f1-score   support

not psychopathic       0.75      0.75      0.75         4
    psychopathic       0.83      0.83      0.83         6

        accuracy                           0.80        10
       macro avg       0.79      0.79      0.79        10
    weighted avg       0.80      0.80      0.80        10



## Binary - KFold

In [ ]:
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.feature_extraction.text import CountVectorizer

# Initialize KFold
kf = KFold(n_splits=5)

# Prepare the classifier
clf = LogisticRegression()

# Convert text data to numerical data
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(df['clean_text'])
y = df['binary']

# Store the accuracy for each fold and predictions
accuracies = []
all_y_test = []
all_y_pred = []

for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Train the model
    clf.fit(X_train, y_train)

    # Make predictions
    y_pred = clf.predict(X_test)

    # Store true labels and predictions
    all_y_test.extend(y_test)
    all_y_pred.extend(y_pred)

    # Calculate accuracy
    accuracy = accuracy_score(y_test, y_pred)
    accuracies.append(accuracy)

# Calculate the average accuracy
average_accuracy = sum(accuracies) / len(accuracies)
print(f"Average Accuracy: {average_accuracy}")

# Generate classification report
print("Classification Report:\n")
print(classification_report(all_y_test, all_y_pred))




Average Accuracy: 0.7
Classification Report:

                  precision    recall  f1-score   support

not psychopathic       0.67      0.50      0.57         4
    psychopathic       0.71      0.83      0.77         6

        accuracy                           0.70        10
       macro avg       0.69      0.67      0.67        10
    weighted avg       0.70      0.70      0.69        10



## TTS - Group Comparison (but have to use 'Handbook' in train because of healthy controls)

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.pipeline import Pipeline

# Function to train with Stratified K-Fold CV based on group, and return the trained model
def train_and_return_model_skf(df, group_names, k=2):
    # Filter DataFrame based on the list of groups
    group_df = df[df['group'].isin(group_names)]

    # Split data into features (X) and target (y)
    X = group_df['clean_text']
    y = group_df['binary']

    # Initialize StratifiedKFold
    skf = StratifiedKFold(n_splits=k)

    accuracies = []

    for train_index, test_index in skf.split(X, y):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        pipeline.fit(X_train, y_train)
        y_pred = pipeline.predict(X_test)
        accuracies.append(accuracy_score(y_test, y_pred))

    average_accuracy = sum(accuracies) / len(accuracies)
    return pipeline, average_accuracy

# Example usage
group_name = ['articles', 'handbook']
model, accuracy = train_and_return_model_skf(df, group_name)
print(f"Average Accuracy for training groups '{group_name}' with Stratified K-Fold CV: {accuracy}")



NameError: ignored

In [ ]:
def get_texts_by_group(df, group_names):
    # Filter DataFrame based on the list of groups
    filtered_df = df[df['group'].isin(group_names)]

    # Return the 'clean_text' column from the filtered DataFrame
    return filtered_df['clean_text'].tolist()

# Example usage with a list of group names
test_names = ['doc', 'cbc']
test_texts = get_texts_by_group(df, test_names)
print(test_texts)


[' my very first contemplation of criminal behavior was trying to uh put together a plan of getting away with murder and i was gonna kill my brother i knew that every sunday id jump on his bike and hed head down the road to join up with some other fellas and he loved that bike and he used to tell me you ever touch my bike ill kill you so this sunday morning while hes working i take his bike i knew if i had provoked it just a little bit more that he would chase me so i gave him a couple of kicks and took off on a dead run for a tree that was right beside my grandfathers barn and it went up onto the roof and on the opposite side of the barn there were cutter rakes and sure enough he chased me up there and as he uh topped edge of that roof i kicked him and he fell onto those rakes some court people came got me i stabbed my first man um i uh stabbed him uh he lived but it sent out a word a clear word to the rest of them that uh you dont wanna be messing with this kid hell stick ya this par

In [ ]:
predictions = model.predict(test_texts)
print(predictions)

NameError: ignored

## Trinary - LOOCV

In [ ]:
import pandas as pd
from sklearn.model_selection import LeaveOneOut
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.feature_extraction.text import CountVectorizer

# Initialize LeaveOneOut
loocv = LeaveOneOut()

# Prepare the classifier
clf = LogisticRegression(max_iter=1000)  # Increase max_iter if needed

# Convert text data to numerical data
vectorizer = CountVectorizer()

X = vectorizer.fit_transform(df['clean_text'])
y = df['trinary']

# Store the accuracy for each fold and predictions
accuracies = []
all_y_test = []
all_y_pred = []

for train_index, test_index in loocv.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Train the model
    clf.fit(X_train, y_train)

    # Make predictions
    y_pred = clf.predict(X_test)

    # Store true labels and predictions
    all_y_test.extend(y_test)
    all_y_pred.extend(y_pred)

    # Calculate accuracy
    accuracy = accuracy_score(y_test, y_pred)
    accuracies.append(accuracy)

# Calculate the average accuracy
average_accuracy = sum(accuracies) / len(accuracies)
print(f"Average Accuracy: {average_accuracy}")

# Generate classification report
print("Classification Report:\n")
print(classification_report(all_y_test, all_y_pred))




Average Accuracy: 0.7
Classification Report:

              precision    recall  f1-score   support

        high       0.83      0.83      0.83         6
         low       0.00      0.00      0.00         1
      medium       0.50      0.67      0.57         3

    accuracy                           0.70        10
   macro avg       0.44      0.50      0.47        10
weighted avg       0.65      0.70      0.67        10



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Trinary - Kfold

In [ ]:
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.feature_extraction.text import CountVectorizer

# Initialize KFold
kf = KFold(n_splits=5)

# Prepare the classifier
clf = LogisticRegression()

# Convert text data to numerical data
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(df['clean_text'])
y = df['trinary']  # Change this to the column with three categories

# Store the accuracy for each fold and predictions
accuracies = []
all_y_test = []
all_y_pred = []

for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Train the model
    clf.fit(X_train, y_train)

    # Make predictions
    y_pred = clf.predict(X_test)

    # Store true labels and predictions
    all_y_test.extend(y_test)
    all_y_pred.extend(y_pred)

    # Calculate accuracy
    accuracy = accuracy_score(y_test, y_pred)
    accuracies.append(accuracy)

# Calculate the average accuracy
average_accuracy = sum(accuracies) / len(accuracies)
print(f"Average Accuracy: {average_accuracy}")

# Generate classification report
print("Classification Report:\n")
print(classification_report(all_y_test, all_y_pred))


Average Accuracy: 0.5
Classification Report:

              precision    recall  f1-score   support

        high       0.62      0.83      0.71         6
         low       0.00      0.00      0.00         1
      medium       0.00      0.00      0.00         3

    accuracy                           0.50        10
   macro avg       0.21      0.28      0.24        10
weighted avg       0.38      0.50      0.43        10



/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

## TTS - All

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.feature_extraction.text import CountVectorizer

# Load your dataset (assuming df is already loaded)
# df = pd.read_csv('your_dataset.csv')

# Convert text data to numerical data
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(df['clean_text'])
y = df['binary']

# Split the data into 80% training and 20% testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Define the hyperparameter grid for the LogisticRegression model
param_dist = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100],
    'max_iter': [100, 500, 1000, 1500],
}

# Initialize the classifier
clf = LogisticRegression()

# Initialize RandomizedSearchCV
random_search = RandomizedSearchCV(
    clf, param_distributions=param_dist, n_iter=10, cv=4, scoring='accuracy', random_state=42)

# Fit RandomizedSearchCV to find the best hyperparameters
random_search.fit(X_train, y_train)

# Get the best estimator with the optimized hyperparameters
best_clf = random_search.best_estimator_

# Train the best model on the full training set
best_clf.fit(X_train, y_train)

# Make predictions on the test set
y_pred = best_clf.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy on the test set: {accuracy}")

# Generate classification report
print("Classification Report:\n")
print(classification_report(y_test, y_pred))


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=4.
  warnings.warn(


Accuracy on the test set: 1.0
Classification Report:

                  precision    recall  f1-score   support

not psychopathic       1.00      1.00      1.00         1
    psychopathic       1.00      1.00      1.00         2

        accuracy                           1.00         3
       macro avg       1.00      1.00      1.00         3
    weighted avg       1.00      1.00      1.00         3

